In [1]:
!pip install pandas numpy sklearn openai


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
# Install Kaggle library
!pip install kaggle

# Make Kaggle directory and move the API key
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Set correct permissions for the key
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [3]:
# Download the dataset
!kaggle datasets download -d tobiasbueck/multilingual-customer-support-tickets

# Unzip the dataset
!unzip multilingual-customer-support-tickets.zip


Dataset URL: https://www.kaggle.com/datasets/tobiasbueck/multilingual-customer-support-tickets
License(s): Attribution 4.0 International (CC BY 4.0)
  0% 0.00/2.08M [00:00<?, ?B/s]
100% 2.08M/2.08M [00:00<00:00, 128MB/s]
Archive:  multilingual-customer-support-tickets.zip
  inflating: dataset-tickets-multi-lang3-4k.csv  


In [8]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import openai
import re

# Load Dataset from Kaggle
file_path = '/content/multilingual-customer-support-tickets.zip'
df = pd.read_csv(file_path)

# Filter English tickets
df = df[df['language'] == 'en']

# Preview dataset
print(df.head())


                                              subject  \
3             Urgent Assistance Required: AWS Service   
5      Urgent: Issue with Zoom Screen Sharing Feature   
7   Service outage resolution requested for ongoin...   
8                                 Surface Pro 7 Issue   
10  Request for software development consultation ...   

                                                 body  \
3   Dear IT Services Support Team, \n\nI am reachi...   
5   Dear Customer Support,\n\nI am experiencing a ...   
7   Dear IT Services Customer Support,\n\nI am wri...   
8   Dear Tech Online Store Support Team,\n\nI hope...   
10  Dear Customer Support,\n\nI hope this message ...   

                                               answer      type  \
3   Dear <name>,\n\nThank you for reaching out reg...   Request   
5   Dear <name>,\n\nThank you for reaching out. We...  Incident   
7   Dear <name>,\n\nThank you for reaching out to ...    Change   
8   Dear <name>,\n\nThank you for reaching out

In [18]:
# Text Preprocessing
def preprocess_text(text):
    # Handle None and non-string types
    if text is None or not isinstance(text, str):
        return ''  # Return empty string instead of None

    text = text.lower()
    text = re.sub(r'\W', ' ', text)  # Remove non-word characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text.strip()

# Apply preprocessing and create 'clean_text' column from 'subject' column
df['clean_text'] = df['subject'].apply(preprocess_text) # Changed 'text' to 'subject' to apply preprocessing on the correct column

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Convert text to numerical data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['clean_text'])

# Apply KMeans Clustering
num_clusters = 5  # Adjust based on dataset size
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

# Display Clustered Issues
for i in range(num_clusters):
    print(f"\nCluster {i}:")
    print(df[df['cluster'] == i]['subject'].head(3)) # Changed 'text' to 'subject' to display the original subject text


Cluster 0:
5       Urgent: Issue with Zoom Screen Sharing Feature
7    Service outage resolution requested for ongoin...
8                                  Surface Pro 7 Issue
Name: subject, dtype: object

Cluster 1:
26     Inquiry About Dell XPS 13 Battery Optimization
73    Exchange Request for Malfunctioning Dell XPS 13
89              Inquiry about Dell XPS 13 9310 issues
Name: subject, dtype: object

Cluster 2:
3               Urgent Assistance Required: AWS Service
16    Immediate Assistance Required for Network Disr...
19                 Assistance Needed for Chrome Crashes
Name: subject, dtype: object

Cluster 3:
48     Network Slow Connectivity Issue Reported with ...
66                                   Urgent: Router Down
120    High Priority Incident: Cisco Router ISR4331 F...
Name: subject, dtype: object

Cluster 4:
57                   Issue with HP DeskJet 3755 Printer
85     Connectivity Issues with HP DeskJet 3755 Printer
197            Connectivity Issues with HP Des

In [20]:
# Define templates based on identified categories
category_templates = {
    'payment_issue': "Dear customer, it seems you're facing a payment issue. Please follow these steps: XYZ.",
    'login_issue': "Dear customer, it looks like you're having trouble logging in. Follow these steps: ABC.",
    'general_inquiry': "Dear customer, we have received your inquiry. Our team will get back to you shortly.",
}

# Save key variables in a dictionary
issue_data = {
    'payment_issue': {'steps': "Check your payment method, ensure sufficient funds, and retry."},
    'login_issue': {'steps': "Reset your password, clear browser cache, and retry."}
}


In [21]:
def classify_and_respond(user_query):
    # Preprocess the query
    user_query = preprocess_text(user_query)
    user_vector = vectorizer.transform([user_query])

    # Identify cluster
    cluster = kmeans.predict(user_vector)[0]
    cluster_texts = df[df['cluster'] == cluster]['clean_text'].tolist()

    # Find closest match using cosine similarity
    similarities = cosine_similarity(user_vector, vectorizer.transform(cluster_texts))
    closest_idx = np.argmax(similarities)
    matched_text = cluster_texts[closest_idx]

    # Map to issue category
    if 'payment' in matched_text:
        template = category_templates['payment_issue']
        steps = issue_data['payment_issue']['steps']
    elif 'login' in matched_text:
        template = category_templates['login_issue']
        steps = issue_data['login_issue']['steps']
    else:
        template = category_templates['general_inquiry']
        steps = ""

    # Generate final response
    response = template.replace('XYZ', steps).replace('ABC', steps)
    return response

# Test the pipeline
user_input = "I can't make a payment on your website."
print(classify_and_respond(user_input))


Dear customer, it seems you're facing a payment issue. Please follow these steps: Check your payment method, ensure sufficient funds, and retry..


In [23]:
import joblib

# Save KMeans Model and Vectorizer
joblib.dump(kmeans, 'kmeans_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')



['tfidf_vectorizer.pkl']

In [24]:
queries = [
    "I'm facing payment issues.",
    "I can't log in to my account.",
    "I have a general question about your services."
]

for query in queries:
    print(f"User Query: {query}")
    print(f"Response: {classify_and_respond(query)}\n")


User Query: I'm facing payment issues.
Response: Dear customer, it seems you're facing a payment issue. Please follow these steps: Check your payment method, ensure sufficient funds, and retry..

User Query: I can't log in to my account.
Response: Dear customer, we have received your inquiry. Our team will get back to you shortly.

User Query: I have a general question about your services.
Response: Dear customer, we have received your inquiry. Our team will get back to you shortly.

